In [2]:
# imports 
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from shapely.geometry import mapping
import pandas as pd



In [28]:
# load the shapefile
file = "output\\rehab_poly_exploded\\rehab_poly_exploded.shp"
# file = "output\\rehab_poly_unmodified\\rehab_poly_unmodified.shp"
shapes = gpd.read_file(file)
shapes = shapes.to_crs("EPSG:7856")  # match CHM CRS


In [10]:
# Open CHM raster and mask it

chm_file = "output\\chm_0.2.tif"

with rasterio.open(chm_file) as src:
    chm_meta = src.meta.copy()

    # Clip to outer boundary of all polygons
    out_image, out_transform = mask(src, shapes.geometry, crop=True)
    chm_clipped = out_image[0]  # first band

In [ ]:
# group by MAP_NAME and calculate statistics
results = []

with rasterio.open(chm_file) as src:
    for _, row in shapes.iterrows():
        geom = [mapping(row.geometry)]
        name = row["MAP_NAME"]

        try:
            out_image, _ = mask(src, geom, crop=True)
            chm = out_image[0]
            chm = chm[chm > 0]  # remove NoData or zero-height

            if chm.size > 0:
                result = {
                    "MAP_NAME": name,
                    "area_m2": chm.size * src.res[0] * src.res[1],
                    "mean_height_m": np.mean(chm),
                    "p90_height_m": np.percentile(chm, 90),
                    "woody_cover_pct": np.mean(chm > 1.0) * 100,
                }
                results.append(result)

        except ValueError:
            # geometry might be outside raster bounds
            continue

In [12]:
df = pd.DataFrame(results)
df.sort_values("MAP_NAME", inplace=True)

df

,MAP_NAME,area_m2,mean_height_m,p90_height_m,woody_cover_pct
27,2015 Rehab,1872.24,1.218871,3.949995,31.995898
29,2015 Rehab,62268.20,1.870076,5.190002,46.747907
30,2015 Rehab,41.64,2.592531,5.469996,58.981748
31,2015 Rehab,50917.60,1.484131,4.790002,36.316244
32,2015 Rehab,44883.60,1.840981,5.129998,45.076063
28,2015 Rehab,10201.76,1.242594,3.895588,27.797949
33,2016 Rehab,688.20,0.294406,1.029999,10.915432
34,2016 Rehab,112.20,0.280608,0.702029,5.989305
35,2016 Rehab,186.00,0.273233,1.010007,10.322581
36,2016 Rehab,25.08,0.805574,1.825248,26.475279


In [30]:
# extract all metadata fields
# shapes['ID']
shapes

,LAYER,MAP_NAME,CLOSED,BORDER_STY,BORDER_COL,BORDER_WID,FILL_STYLE,FILL_COLOR,Area,UNIQUE_ID,rehab_year,veg_type,retro_tmp,notes,veg_method,rehab_zone,geometry
0,Unknown Area Type,2020 Retrofit Tubestock,YES,Solid,"RGB(0,255,0)",2,No Fill,None,12.467510,0,2020.0,hdwv,y,HDWV Tube Stock May 2020,tubestock,4a,"POLYGON ((297089.5 6428110.42, 297147.62 64281..."
1,Unknown Area Type,2021 HDWV,YES,Solid,"RGB(255,0,0)",2,No Fill,None,32.771808,1,2021.0,hdwv,n,None,seed,7,"POLYGON ((296910.813 6428140.195, 296909.873 6..."
2,Unknown Area Type,2021 HDWV,YES,Solid,"RGB(255,0,0)",2,No Fill,None,32.771808,1,2021.0,hdwv,n,None,seed,7,"POLYGON ((295747.388 6426438.373, 295747.238 6..."
3,Unknown Area Type,2021 HDWV,YES,Solid,"RGB(255,0,0)",2,No Fill,None,32.771808,1,2021.0,hdwv,n,None,seed,7,"POLYGON ((296148.837 6427261.672, 296148.537 6..."
4,Unknown Area Type,2021 HDWV,YES,Solid,"RGB(255,0,0)",2,No Fill,None,32.771808,1,2021.0,hdwv,n,None,seed,None,"POLYGON ((296078.538 6427421.832, 296078.348 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Unknown Area Type,2020 Retrofit HDWV,YES,Solid,"RGB(0,0,0)",1,Solid Fill,"RGB(255,255,255)",0.033275,51,2020.0,hdwv,y,None,seed,None,"POLYGON ((296895.004 6427793.646, 296928.353 6..."
71,Unknown Area Type,2020 Retrofit HDWV,YES,Solid,"RGB(0,0,0)",1,Solid Fill,"RGB(255,255,255)",0.036740,52,2020.0,hdwv,y,None,seed,None,"POLYGON ((296993.493 6427979.135, 296960.723 6..."
72,Unknown Area Type,2020 Retrofit HDWV,YES,Solid,"RGB(0,0,0)",1,Solid Fill,"RGB(255,255,255)",0.236036,53,2020.0,hdwv,y,None,seed,None,"POLYGON ((297016.953 6428065.384, 297038.472 6..."
73,Unknown Area Type,2020 Retrofit HDWV,YES,Solid,"RGB(0,255,0)",2,No Fill,None,14.961322,54,2020.0,hdwv,y,None,seed,6,"POLYGON ((296049.259 6426121.604, 296034.219 6..."
